# Processing pipeline ISLAND MRI examcard

**Code version**: 2.0 (Nov. 2021)  
**Code by:**  

    Max Keuken,
    University of Amsterdam, The Netherlands
    mckeuken@gmail.com
    
**Goal of project and code:**  

    In collaboration with Mark Hinder, Jane Alty, Jak Ma Wen, Sarah Kemp, Pilou Bazin and Birte Forstmann 
    we tried to come up with a flexible 3T MRI exam card that could potentially be used as part of the 
    ISLAND study that is being done in Tasmania Australia. 
    
    The main goal of the exam card is that should provide isotropic, multiparameter, quantitative maps that
    can be acquired in a short amount of time. 
    
    The exam card consists of the following sequences:
    - mp2rage (which delivers T1w and T1-maps)
    - 3DEPI (which delivers SWI and QSM)
    - DWI (30 directions which delivers ADC, AD, FA, RD)
    - ASL
    
    Given the nature of the sequences a bit of processing will need to be done to actually get the quantitative 
    maps that are all in the same space. This is what this collection of notebooks are for.
    
**Data format**

    Each subject's acquired exam card is exported as a DICOM
    Each subjects DICOM folder has the following foldername structure:
    ISLAND_sub-0001_ses-1 (for the first session of subject 1 within the island study)
    ISLAND_pilot-001 (for the first pilot subject within the island study)

**Software requirements:**
    
    Initially the notebook was written on a macbook pro, M1 core (2020) but the QSM processing 
    needed substantially more ram memory (+/- 20gb). The code is now written for a linux server
    where you have the following software installed at the terminal command line level: 
    dcm2nix v1.0.20190902
    mrtrix3 v3.02
    fsl v5
    ants v2.3.5
    python v3.7.7
    singularity V3+ (https://sylabs.io)
    
    Once you have singularity installed, you need to install the images via singularity:
    QSM related: https://github.com/CAIsr/qsm

    It needs to work via the terminal command line, which can be tested by typing in
    tgv_qsm 
    
    I would also recommend that you install the jupyter lab extension '@jupyterlab/toc' as this
    allows you to go through the code much easier using the table of content tab at the left side
    of your jupyterlab interface
        
**MNI Atlas requirement**
    
    Basically any atlas that is in standard space can be used pretty easily in this pipeline. The
    only thing that you have to make sure is that your atlas has the following things:
    - it needs to have the exact same dimensions as the MNI template used in this pipeline. Even 
         when it is a MNI based atlas the dimension of the file can still be different. This can
         be solved by running a fix registration step using FSL FLIRT. The actual code to do so is 
         given in the section where the MNI atlas is used.
    - it needs to have a text file that per row has the label of the structure 
    - it needs the actual atlas which should be an nifti file where every single integer value
        corresponds to a mask. Also note that the order of anatomical labels in the text file 
        and the nifti file need to be IDENTICAL. 
    
**How to run this notebook?**

    If you have never worked with jupyter notebooks / jupyter labs, have a look at https://ipython-books.github.io/
    tldr: select a cell that you want to run and press 'shift+enter' on a mac/linux and you will run the block of code
    
**Status of code**

- [x] modules
- [x] paths
- [x] external files
- [x] dicom 2 nifti
- [x] mp2rage
- [x] 3DEPI
- [x] DWI
- [x] ASL
- [x] registration to mp2rage space
- [X] registration to mni space
- [X] extract the qMRI values per atlas region and export to excel 

## Setup the current enviroment

### Loading in the required python modules

In [ ]:
#################################################################################
# Python modules
#################################################################################
#. Note that if you are missing one of the following packages you can install 
#. it via 'pip install 'package name'' on the command line

####
# standard python modules
###
import shutil, sys, os, re, subprocess, json, numpy as np, pandas as pd, glob as glob, warnings

# Suppress all warnings
#. If you are debugging you might want to turn this back on again
warnings.filterwarnings("ignore")

####
# MRI related python modules
####
#. We are using the pymp2rage python package which is programmed by Gilles de Hollander
#. and is based on the paper by Jose Marques et al. (2010) neuroimage. 
#. 
#. You can install this package by typing pip install git+https://github.com/mckeuken/pymp2rage
#. The original package is found here: https://github.com/Gilles86/pymp2rage/ but contains a typo
#. in the setup file which interferes with the installation. Hence my own fork.
#. See https://github.com/Gilles86/pymp2rage/issues/3 for the description of the typo. 
import pymp2rage, nibabel as nib, bids as bids
import nipype.interfaces.fsl as fsl
from nipype.interfaces.ants import RegistrationSynQuick
from nipype.interfaces.ants import ApplyTransforms
import ants

####
# Visualisation related python modules
####
import matplotlib.pyplot as plt, seaborn as sns
from nilearn import plotting
%matplotlib inline

print('All modules have been imported')
#################################################################################

### Setting the paths and determining the subject names

In [ ]:
#################################################################################
# Path settings and subject names
#################################################################################
#. In the end we want to have a folder that is BIDS compatible as this will
#. make your life a lot easier when dealing with published pipelines and it 
#. also give you a good folder structure to work with. In general we want 
#. to have a layout where we have the following:
#. - subject
#. -- session <- necessary because the island study is longitudinal
#. --- anatomy folder that contains the t1 and t2* related data
#. --- diffusion weighted data folder
#. --- perfusion weighted data folder
#. --- dicom 2 nifti folder that contains all the raw exported scans

####
# Set main project folder 
#. NOTE: These folders need to exist already 
projectDir = '/home/mkeuken1/Documents/Projects/TasScans' # <- !!!CHANGE BASED ON OWN SERVER!!!
syntaxDir = projectDir+'/syntax'
rawDataDir = projectDir+'/original/dicom' # plus a session folder: 'pilot', 'ses-0001', 'ses-0002', etc
processedDataDir = projectDir+'/data'

# Location of the MNI template and the atlas used
#. This is the standard 1mm MNI template as is incorporated within FSL
mniTemplate =  syntaxDir+'/avg152T1_brain.nii.gz'       
mniAtlasLabels = syntaxDir+'/MNIAtlas/BN_Atlas_246_LUT.txt'
mniAtlas = syntaxDir+'/MNIAtlas/BN_Atlas_246_1mm.nii'

####
#. Which acquisition session are we processing?
session = 'pilot' # Options: 'pilot', 'ses-0001', 'ses-0002', etc

####
# Get all the subject IDs from the dicom folders
# Step 1) list all the subject folders and transform them to lowercase
subjects_dicom = glob.glob(rawDataDir+'/'+session+'/*')
subjects_Tmp = [item.lower() for item in subjects_dicom]

# Step 2) Rename the subject lists to match the BIDS format:
subjects = []
for i in range(len(subjects_Tmp)):
    # Split the path into the different folders
    foldername = subjects_Tmp[i].split('/') 
    # Get the numbers as written down in the subject (last) folder:
    numericsInFoldername = re.findall(r'\d+', foldername[-1])[0]
    # Add the number to the string 'sub' while padding the number of zero's
    #. note that it goes upto 9999 subjects. If you need a wider range change 
    #. the 4 to say 5 (and thus 99999) subjects
    bidsName ='sub-'+str(numericsInFoldername).zfill(4)
    subjects.append(bidsName)
# Lets also sort the subjects so that it is a bit nicer
subjects_dicom.sort()
subjects.sort()
# Print the number of subjects that we now have
print('The number of subjects for which we have MRI data of this session is: ', len(subjects))
#################################################################################

### Defining Custom config files

In [ ]:
#################################################################################
# Custom config files
#################################################################################
#. For the DWI we are acquiring extra files so that we can apply topup. For that
#. you need a topup configuration file that is part of fsl or you can specify 
#. one yourselve which we do in our case. 

# This config file is an ALTERED VERSION OF FSL's b02b0.cnf and can be used for higher resolution data.
# The file itself has the following information:
customb02b0 = """# Resolution (knot-spacing) of warps in mm
--warpres=20,16,14,12,10,6,4,4,4,2.5
# Subsampling level (a value of 2 indicates that a 2x2x2 neighbourhood is collapsed to 1 voxel)
--subsamp=2,2,2,2,2,1,1,1,1,1
# FWHM of gaussian smoothing
--fwhm=8,6,4,3,3,2,1,0,0,0
# Maximum number of iterations
--miter=5,5,5,5,5,10,10,20,20,25
# Relative weight of regularisation
--lambda=0.005,0.001,0.0001,0.000015,0.000005,0.0000005,0.00000005,0.0000000005,0.00000000001,0.00000000001
# If set to 1 lambda is multiplied by the current average squared difference
--ssqlambda=1
# Regularisation model
--regmod=bending_energy
# If set to 1 movements are estimated along with the field
--estmov=1,1,1,1,1,0,0,0,0,0
# 0=Levenberg-Marquardt, 1=Scaled Conjugate Gradient
--minmet=0,0,0,0,0,1,1,1,1,1
# Quadratic or cubic splines
--splineorder=3
# Precision for calculation and storage of Hessian
--numprec=double
# Linear or spline interpolation
--interp=spline
# If set to 1 the images are individually scaled to a common mean intensity 
--scale=1"""

###
# Create the b02b0_hires.cnf file:
with open(syntaxDir+'/b02b0_hires.cnf', 'w+') as f:
    f.writelines(customb02b0)
    
print('Custom b02b0 file has been created')
#################################################################################

## Extract the NIfTI files from the raw dicoms

In [ ]:
#################################################################################
## 1. Convert raw data to NIfTI using dcm2niix
#################################################################################
####
#. If you want to rerun the code (and thus OVERWRITE your previous results),
#. set overwrite to True
overwrite = True
####

####
# Start the loop to export the dicom data to nifti
####
#. Lets start with an empty list for summary info:
summaryScans = []

#. after which we can start the for loop to create a folder per subject
for i in range(len(subjects)):
    print('\n', subjects[i])
    #. BIDS'ish layout: subject, session, modality, data
    #. Where we make a seperate folder for pilot data and final data in the ISLAND folder
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
    
    dicom_dir = subjects_dicom[i]
    convert_dir = work_dir+'/dcm2nii' # This will contain all the exported NIfTI files from all scans
    anat_dir = work_dir+'/anat' # which will contain the T1 and T2* derived images
    dwi_dir = work_dir+'/dwi' # this will contain all the DWI derived images
    
    # Create folder and convert dicom to nifti
    if ((os.path.exists(convert_dir)) & (overwrite == False)):
        print('Conversion to NIfTI: done (set overwrite to True to recompute)')
    else:
        print('\n Converting Dicom into NIfTI files')
        try: 
            shutil.rmtree(convert_dir)
            os.makedirs(convert_dir)
        except:
            os.makedirs(convert_dir)
        command = "dcm2niix -o "+convert_dir+' -z y '+dicom_dir
        print(command)
        try:
            subprocess.check_output(command, shell = True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        #. As you can see in the folder output, the exam card has quite a lot of extra scans that we 
        #. didn't really acquire. Instead the same data is exported multiple times and includes
        #. other files that were derived from the scanner console. We are going to
        #. clean up the output folder by making a folder of scansOfNoInterest which will include the
        #. data that is exported twice.
        #. First make the folder
        scansOfNoInterest_dir = convert_dir+'/scansOfNoInterest'
        if not os.path.exists(scansOfNoInterest_dir):
            os.makedirs(scansOfNoInterest_dir)
        else:
            shutil.rmtree(scansOfNoInterest_dir)          
            os.makedirs(scansOfNoInterest_dir)

        #. Which files do we want to move?
        #. Any files that have a file counter in there that is larger than 99
        scansThatWeWantToMove = glob.glob(convert_dir+'/*.*' , recursive=False)
        for scan in scansThatWeWantToMove:
            fileName = scan.split('/')
            examCardCounter = re.findall(r'\d+', fileName[-1])[-1]
            if int(examCardCounter) > 99:
                 shutil.move(scan, scansOfNoInterest_dir)

        # We also want to move the files that were derived at the scanner console
        #. We can find this information in the json file, field 'ImageType'
        # First make the folder:
        consoleDerived_dir = convert_dir+'/consoleDerived'
        if not os.path.exists(consoleDerived_dir):
            os.makedirs(consoleDerived_dir)
        else:
            shutil.rmtree(consoleDerived_dir)          
            os.makedirs(consoleDerived_dir)
            
        #. Which files do we want to move?
        #. Find the json files that we still have in the main folder
        scansThatWeWantToMove = glob.glob(convert_dir+'/*.json' , recursive=False)
        #. Open the actual json file and check whether the string 'DERIVED'
        #. occures in the field 'ImageType'. Ifso, then move all files with
        #. all the different extensions to the console derived folder
        for jsonFile in scansThatWeWantToMove:
            tmpJson = open(jsonFile)
            JsonData = json.load(tmpJson)
            if 'DERIVED' in JsonData['ImageType']:
                moveFiles = glob.glob(jsonFile.rsplit( ".", 1 )[ 0 ]+'.*' , recursive=False)
                for file in moveFiles:
                    shutil.move(file, consoleDerived_dir)
            
    #. Get some basic information from the different files
    tmpScans = glob.glob(convert_dir+'/*.nii.gz' , recursive=False)

    #. For the given scans 
    for scan in tmpScans:
        #. Load the image
        dwi_img = nib.load(scan)
        #. Get the actual data
        dwi_img_data = dwi_img.get_fdata()

        #. Get some summary stats on the image and combine them in the list:
        dwi_img_shape = dwi_img_data.shape
        dwi_img_voxelSize = nib.affines.voxel_sizes(dwi_img.affine)
        summaryScans.append([subjects[i], scan.rpartition('/')[-1], dwi_img_shape, np.round(dwi_img_voxelSize, 2)])

#. Format the summarylist into a dataframe: 
dfSummaryScans = pd.DataFrame(summaryScans, columns=['Subject', 'Scan', 'Shape', 'VoxelSize'])
dfSummaryScans = dfSummaryScans.sort_values(by = ['Subject', 'Scan']).reset_index(drop = True)

#. Based on the exam card we expect to have mp2rage (4x) related files, 3DEPI (2x) related files 
#. and DWI related files
display(dfSummaryScans)
#################################################################################

### Create brain masks

#### MP2RAGE scans

In [ ]:
#################################################################################
#. Skull strip the mp2rage INV2 image using BET and only keep the mask
#################################################################################
# This part is iteratively done as betting brains is not trival and needs 
#. quite some custom work. The benefit is that the better you do it now, the 
#. easier (and probably better) the registration between scans and mni space
#. is going to be later on.

# Also note that you will need to do this EVERY session again. The code assumes
#. that you will run your analysis per session.

overwrite = False
# List of subjects for which betting is done:
#. (this list will have to be made empty again once you start analyzing a 
#. new session)
subjectsDone = [] # For example if we have two subject that we are happy with we add them to the list like this: ['sub-0007', 'sub-0013']

# We don't want to have any plotting warnings that have nothing to do with 
#. the actual betting:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # Start the betting per subject
    for i in range(0, len(subjects)):
        print('\n', subjects[i])
        # select the right folder
        if session == 'pilot':
            work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
        else:
            work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        convert_dir = work_dir+'/dcm2nii'
        bet_dir = work_dir+'/bet'
        
        # Check if this subject is already done or not (if not create folder)
        if ((os.path.exists(bet_dir)) & (overwrite == False) & (subjects[i] in subjectsDone)):
            print('BET subject: done (set overwrite to true to recompute AND remove from list subjectsDone )')
        else:
            print('\n BET subject')
            try: 
                shutil.rmtree(bet_dir)
                os.makedirs(bet_dir)
            except:
                os.makedirs(bet_dir)

            # Location of magnitude INV2 file
            anat_t1_related_files = glob.glob(convert_dir+'/*.json' , recursive=False)
            for jsonFile in anat_t1_related_files:
                fileName = jsonFile.split('.')[0]
                tmpJson = open(jsonFile)
                jsonData = json.load(tmpJson, strict = False)
                if 'INV2' in jsonData['SeriesDescription']:
                    if (('NORM' in jsonData['ImageType']) | ('M' in jsonData['ImageType'])):
                        inv2_mag = fileName+'.nii.gz'

            # Bet the magnitude image using BET and multiple iterations:
            # f: fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates
            # g: vertical gradient in fractional intensity threshold (-1->1); default=0; 
            #.    positive values give larger brain outline at bottom, smaller at top

            #. The input files for the betting procedure
            in_file = inv2_mag
            out_file = bet_dir+'/'+subjects[i]+'_'+session+'_brain_T1w-INV2.nii.gz'
            out_file_mask = bet_dir+'/'+subjects[i]+'_'+session+'_brain_T1w-INV2_mask.nii.gz'
            
            ####
            # The betting command that you will need to tweak:
            ####
            command = 'bet '+in_file+' '+out_file+' -f 0.62 -g 0.15 -m -R'
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
                raise subprocess.CalledProcessError(msg)

            # Plot the data to see if the betting worked
            #. If the mask did seem to capture the entire brain and did not remove too much 
            #. from the brainstem, cerebellum, temporal poles etc add that subject's id to 
            #. the list 'subjectsDone' above in the following manner: 
            #.    subjectsDone = ['sub-0001', 'sub-0002'] etc
            fig = plt.figure(figsize=(20, 6))
            plotting.plot_roi(out_file_mask, 
                              bg_img=in_file, 
                              figure=fig,
                              cmap = 'Reds',
                              alpha = 0.3,
                              cut_coords=(0, 0, 0), 
                              draw_cross= False,
                              title = subjects[i] )
            
            # Clean the bet folder so that we only keep the mask:
            os.remove(bet_dir+'/'+subjects[i]+'_'+session+'_brain_T1w-INV2.nii.gz')
#################################################################################

#### 3DEPI scans

In [ ]:
#################################################################################
#. Skull strip the 3D EPI magnitude image using BET and only keep the mask
#################################################################################
# This part is iteratively done as betting brains is not trival and needs 
#. quite some custom work. The benefit is that the better you do it now, the 
#. easier (and probably better) the registration between scans and mni space
#. is going to be later on. 

# Also note that you will need to do this EVERY session again. The code assumes
#. that you will run your analysis per session

# List of subjects for which betting is done:
#. (this list will have to be made empty again once you start analyzing a 
#. new session)
subjectsDone = [] # For example if we have two subject that we are happy with we add them to the list like this: ['sub-0007', 'sub-0013']

# We don't want to have any plotting warnings that have nothing to do with 
#. the actual betting:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # Start the betting per subject
    for i in range(0, len(subjects)):
        print('\n', subjects[i])
        # select the right folder
        if session == 'pilot':
            work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
        else:
            work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        convert_dir = work_dir+'/dcm2nii'
        bet_dir = work_dir+'/bet'
        
        # Check if this subject is already done or not (if not create folder)
        if ( subjects[i] in subjectsDone):
            print('BET subject: done (set overwrite to true to recompute AND remove from list subjectsDone )')
        else:
            print('\n BET subject')
            # Location of magnitude, phase and json files
            anat_qsm_related_files = glob.glob(convert_dir+'/*.json' , recursive=False)
            for jsonFile in anat_qsm_related_files:
                fileName = jsonFile.split('.')[0]
                tmpJson = open(jsonFile)
                jsonData = json.load(tmpJson, strict=False)
                if '3DEPI' in jsonData['SeriesDescription']:
                    if (('NORM' in jsonData['ImageType']) | ('M' in jsonData['ImageType'])):
                        chimap_mag = fileName+'.nii.gz'
        
            # Bet the magnitude image using BET and multiple iterations:
            # f: fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates
            # g: vertical gradient in fractional intensity threshold (-1->1); default=0; 
            #.    positive values give larger brain outline at bottom, smaller at top

            #. The input files for the betting procedure
            in_file = chimap_mag
            out_file = bet_dir+'/'+subjects[i]+'_'+session+'_brain_Chimap.nii.gz'
            out_file_mask = bet_dir+'/'+subjects[i]+'_'+session+'_brain_Chimap_mask.nii.gz'
            
            ####
            # The betting command that you will need to tweak:
            ####
            command = 'bet '+in_file+' '+out_file+' -f 0.4 -g 0.2 -m -R'
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
                raise subprocess.CalledProcessError(msg)

            # Plot the data to see if the betting worked
            #. If the mask did seem to capture the entire brain and did not remove too much 
            #. from the brainstem, cerebellum, temporal poles etc add that subject's id to 
            #. the list 'subjectsDone' above in the following manner: 
            #.    subjectsDone = ['sub-0001', 'sub-0002'] etc
            fig = plt.figure(figsize=(20, 6))
            plotting.plot_roi(out_file_mask, 
                              bg_img=in_file, 
                              figure=fig,
                              cmap = 'Reds',
                              alpha = 0.3,
                              cut_coords=(0, 0, 0), 
                              draw_cross= False,
                              title = subjects[i] )
            
#################################################################################

### Process the MP2RAGE scans

In [ ]:
#################################################################################
# Create T1w, qT1 map
#################################################################################
#. This section creates the unified T1-weighted images and estimates the 
#. quantitative T1 maps. Finally we also create a brain mask based on the 
#. second inversion magnitude image of the mp2rage sequence. 
#. 
#. NOTE: To be able to do all this the magnitude AND phase information is required!
#. Make sure that this data is recorded and exported in the exam card. 

# If you want to overwrite your previous results set overwrite to True. To protect
#. your data that you have calculated before, set overwrite to False
overwrite = False

# Create the model fitter with the specific MRI acquistion parameters per subject
for i in range(0, len(subjects)):
    print('\n', subjects[i])
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        
    convert_dir = work_dir+'/dcm2nii'
    anat_dir = work_dir+'/anat'
    
    if ((os.path.exists(anat_dir)) & (overwrite == False)):
        print('Create T1 unified and T1 map: done (set overwrite to true to recompute)')
    else:
        print('\n Create T1 unified and T1 map')
        try: 
            shutil.rmtree(anat_dir)
            os.makedirs(anat_dir)
        except:
            os.makedirs(anat_dir)
        
        # Location of magnitude, phase and json files
        anat_t1_related_files = glob.glob(convert_dir+'/*.json' , recursive=False)
        
        for jsonFile in anat_t1_related_files:
            fileName = jsonFile.split('.')[0]
            tmpJson = open(jsonFile)
            jsonData = json.load(tmpJson, strict = False)
            if 'INV1' in jsonData['SeriesDescription']:
                if (('NORM' in jsonData['ImageType']) | ('M' in jsonData['ImageType'])):
                    inv1_mag = fileName+'.nii.gz'
                    inv1_json = jsonData
                elif (('PHASE' in jsonData['ImageType']) | ('P' in jsonData['ImageType'])):
                    inv1_ph = fileName+'.nii.gz'
            elif 'INV2' in jsonData['SeriesDescription']:
                if (('NORM' in jsonData['ImageType']) | ('M' in jsonData['ImageType'])):
                    inv2_mag = fileName+'.nii.gz'
                    inv2_json = jsonData
                elif (('PHASE' in jsonData['ImageType']) | ('P' in jsonData['ImageType'])):
                    inv2_ph = fileName+'.nii.gz'

        # Define the mp2rage fitter with the acquisiton parameters from the json file and the number of z slices
        zImage = nib.load(inv1_mag)
        #. Get the data and dimensions of the mp2rage scan:
        zImage = zImage.get_fdata()
        zImage_shape = zImage.shape
        
        # Fit the pymp2rage fitter to the actual data
        #. Note that you might get a warning that states something along the lines of 'True_divide'. 
        #. This is fine and can be ignored.
        fitter = pymp2rage.MP2RAGE(MPRAGE_tr = inv1_json['RepetitionTime'],
                                   invtimesAB = [inv1_json['InversionTime'], inv2_json['InversionTime']],
                                   flipangleABdegree = [inv1_json['FlipAngle'], inv2_json['FlipAngle']],
                                   nZslices = zImage_shape[-1],
                                   FLASH_tr = [inv1_json['EchoTime'], inv2_json['EchoTime']],
                                   B0 = inv1_json['MagneticFieldStrength'],
                                   inv1 = inv1_mag, 
                                   inv1ph = inv1_ph,
                                   inv2 = inv2_mag,                      
                                   inv2ph = inv2_ph)

        # Estimate the Unified T1 weighted image and plot it:
        fig = plt.figure(figsize=(20, 6))
        plotting.plot_anat(fitter.t1w_uni, figure=fig, cut_coords=(0, 0, 0), title = subjects[i] )

        # As the T1 map is quantitative, the values actually mean something. Therefore you can 
        #. add some color and some max values
        fig = plt.figure(figsize=(20, 6))
        plotting.plot_stat_map(fitter.t1map, cmap=plt.cm.viridis, figure=fig, symmetric_cbar=False, vmax=4000, title = subjects[i])

        # Export the T1 map and T1 unified to the local folder as nifti's
        fitter.t1map.to_filename(anat_dir+'/'+subjects[i]+'_'+session+'_T1map.nii.gz')
        fitter.t1w_uni.to_filename(anat_dir+'/'+subjects[i]+'_'+session+'_T1w.nii.gz')
        
        # Also save a json file to the same folder which contains the main T1w parameters as 
        #. based from the INV1 mag file
        with open(anat_dir+'/'+subjects[i]+'_'+session+'_T1w.json', 'w', encoding='utf-8') as f:
            json.dump(inv1_json, f, ensure_ascii=False, indent=4)
        
#################################################################################

### Process the 3D EPI scans into QSM files

In [ ]:
#################################################################################
# Process the 3D EPI data
#################################################################################
overwrite = True

# run the TGV_QSM pipeline on the 3DEPI data
for i in range(0, len(subjects)):
    print('\n', subjects[i])
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        
    convert_dir = work_dir+'/dcm2nii'
    anat_dir = work_dir+'/anat'
    bet_dir = work_dir+'/bet'
    # As the QSM data will end up in the same folder as the T1w data we cannot
    #. check if the folder just exists and thus remove it etc. If we want to rerun 
    #. the QSM part as that would also remove the T1 processed data. We therefore 
    #. need to check if a certain file exists:
    qsmFileExists = anat_dir+'/'+subjects[i]+'_'+session+'_Chimap.nii.gz' # According to the BIDS format: QSM -> Chimap
    
    if ((os.path.exists(dwi_dir+'/')) & (overwrite == False)):
        print('Process the 3D EPI data: done (set overwrite to true to recompute)')
    else:
        print('\n Process the 3D EPI data')
        
        # Location of magnitude, phase and json files
        anat_qsm_related_files = glob.glob(convert_dir+'/*.json' , recursive=False)
        for jsonFile in anat_qsm_related_files:
            fileName = jsonFile.split('.')[0]
            tmpJson = open(jsonFile)
            jsonData = json.load(tmpJson, strict=False)
            if '3DEPI' in jsonData['SeriesDescription']:
                if (('NORM' in jsonData['ImageType']) | ('M' in jsonData['ImageType'])):
                    chimap_mag = fileName+'.nii.gz'
                    chimap_json = jsonData
                elif (('PHASE' in jsonData['ImageType']) | ('P' in jsonData['ImageType'])):
                    chimap_ph = fileName+'.nii.gz'
        
        # Assign the mask that is based on the magnitude to a variable:
        #. This file was created using the betting step in the beginning of the notebook.
        chimap_mag_mask = bet_dir+'/'+subjects[i]+'_'+session+'_brain_Chimap_mask.nii.gz'
        
        # Run the QSM fitting (This might take a while and take up consideral RAM while doing so)
        #. where we provide the different parameters directly from the json file. Note that it 
        #. will automatically use all available cores if OpenMP multithreading is available. 
        #. This is a good thing but might interfer with other users on the same server..
        command = 'tgv_qsm -p '+chimap_ph+' -m '+chimap_mag_mask+' -f '+str(float(chimap_json['MagneticFieldStrength']))+' -t '+str(chimap_json['EchoTime'])+' -s -o _qsm_' 
        print(command)
        try:
            subprocess.check_output(command, shell = True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)
         
        # tgv_qsm outputs the qsm file in the same folder as our input data which is still located in the 
        #. dcm2nii folder. So lets rename to BIDS format and move the output file to the anat folder
        old_name = glob.glob(convert_dir+'/*qsm*', recursive=False)[0]  
        new_name =  anat_dir+'/'+subjects[i]+'_'+session+'_Chimap.nii.gz'
        os.rename(old_name, new_name)
#################################################################################

### Process the DWI scans

In [ ]:
#################################################################################
# Process the DWI data
#################################################################################
# The DWI data will be processed in a way to ensure that it is corrected for noise,
#. eddy current, topup, bias field etc after which a diffusion tensor model 
#. is fit to the cleaned up data. The resulting DTI data is used to extract a number
#. of metrics such as FA, RD, MD, and ADC. 
overwrite = False
saveIntermediatedFiles = True # Note I would always keep the intermediate files

# Process the DWI data
for i in range(0, len(subjects)):
    print('\n', subjects[i])
    
    # Determine the main folders:
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        
    convert_dir = work_dir+'/dcm2nii'
    anat_dir = work_dir+'/anat'
    dwi_dir = work_dir+'/dwi'
    dwiInterim_dir = dwi_dir+'/intermediateFiles'
    # Check if the DWI folder already exists and whether you want to remove it
    #. and rerun it again.
    if ((os.path.exists(dwi_dir)) & (overwrite == False)):
        print('Process the DWI data: done (set overwrite to true to recompute)')
    else:
        print('\n Process the DWI data')
        try: 
            shutil.rmtree(dwiInterim_dir, ignore_errors=True)
            shutil.rmtree(dwi_dir, ignore_errors=True)
            os.makedirs(dwi_dir)
        except:
            os.makedirs(dwi_dir)
            
        # We are also going to make a tmp folder where we are going to save all the
        #. preprocessing steps. You can automatically remove those at the end by 
        #. setting saveIntermediatedFiles to False at the top of this module. Given 
        #. the number of files and filesize you probably want to remove them in the end
        #. but if the output doesnt make any sense I would keep the files so that you 
        #. can check where it goes haywire.
        os.makedirs(dwiInterim_dir)

        ##############################
        # 1. Convert to MRtrix3 format
        print('\n1. Convert Nifti files into Mrtrix3 files')
        ##############################
        # Create a list of different file names based on the DWI files that we have
        #. Note that these are not the B0 scans that we are going to use for topup
        #. (as they don't have a corresponding bvals and bvecs file)
        bval_files = glob.glob(convert_dir+'/*.bval')
        for bval_file in bval_files:
            bvec_file = bval_file.replace('.bval','.bvec')
            data_file = bval_file.replace('.bval','.nii.gz')
            mif_file = bval_file.replace('.bval','.mif.gz')
            mif_file = mif_file.replace(convert_dir, dwiInterim_dir)

            command = 'mrconvert -fslgrad '+bvec_file+' '+bval_file+' '+data_file+' '+mif_file
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
                raise subprocess.CalledProcessError(msg)

        # Convert the topup related files:
        topup_files = glob.glob(convert_dir+'/*_phase_*.nii.gz')
        for topup_file in topup_files:
            data_file = topup_file
            bval_file = 'dummy_topup.bval'
            bvec_file = 'dummy_topup.bvec'
            mif_file = topup_file.replace('.nii.gz','.mif.gz')
            mif_file = mif_file.replace(convert_dir, dwiInterim_dir)

            command = 'mrconvert '+data_file+' '+mif_file
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
                raise subprocess.CalledProcessError(msg)

        ##############################         
        # 2. Concatenate the different DWI files
        print('\n2. Concatenate the DWI files')
        ##############################
        mif_files = glob.glob(dwiInterim_dir+'/*.mif.gz')

        # Empty list of DWI related .mif.gz files
        dwi_files = []
        topup_files = []
        for mif_file in mif_files:
            if mif_file.find('_phase_')>-1:
                topup_files.append(mif_file)
            else:
                dwi_files.append(mif_file)
        # We need to sort them so that the first volume has the same phase encoding
        #. als the main DWI file (important for TOPUP step); 
        #. We are ASSUMING that the main DWI file is AP and the extra phase encoding is PA
        topup_files.sort()

        # Merge the DWI files
        if len(dwi_files) <= 1:
            print('\nThere is only a single DWI file so no need to merge')
            shutil.copy(dwi_files[0], dwiInterim_dir+'/dti_raw.mif.gz')
        else:
            print('\nmerge '+str(len(dwi_files))+' dwi files')
            command = 'mrcat -axis 3 '
            for dwi_file in dwi_files:
                command = command+dwi_file+' '
            command = command+dwiInterim_dir+'/dti_raw.mif.gz'

            print(command)
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
                raise subprocess.CalledProcessError(msg)

        # Merge the topup files
        print('\nmerge '+str(len(topup_files))+' topup files')
        command = 'mrcat -axis 3 '
        for topup_file in topup_files:
            command = command+topup_file+' '
        command = command+dwiInterim_dir+'/dti_topup_raw.mif.gz'

        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        ##############################
        # Denoise the DWI data
        print('\n3. Denoise dwi and topup files')
        ##############################
        command = 'dwidenoise -nthreads 4'
        command = command +' -noise '+dwiInterim_dir+'/dti_noisemap.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_raw.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_denoised.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Creating a (noise) residual
        command = 'mrcalc '+dwiInterim_dir+'/dti_raw.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_denoised.mif.gz'
        command = command +' -subtract '+dwiInterim_dir+'/dti_residuals.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        command = 'dwidenoise -nthreads 4'
        command = command +' -noise '+dwiInterim_dir+'/dti_tp_noisemap.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_topup_raw.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_tp_denoised.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        command = 'mrcalc '+dwiInterim_dir+'/dti_topup_raw.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_tp_denoised.mif.gz'
        command = command +' -subtract '+dwiInterim_dir+'/dti_tp_residuals.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        ##############################
        # Degibbs the data
        print('\n3. Degibbs dwi and topup files')
        ##############################
        # DWI data
        command = 'mrdegibbs -nthreads 4 -axes 0,1'
        command = command +' '+dwiInterim_dir+'/dti_denoised.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_unring.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Converting the output back again to nifti output
        command = 'mrconvert -export_grad_fsl '+dwiInterim_dir+'/dti_unring.bvec'
        command = command +' '+dwiInterim_dir+'/dti_unring.bval'
        command = command +' '+dwiInterim_dir+'/dti_unring.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_unring.nii.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Topup data
        command = 'mrdegibbs -nthreads 4 -axes 0,1'
        command = command +' '+dwiInterim_dir+'/dti_tp_denoised.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_tp_unring.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Converting the output back again to nifti output
        command = 'mrconvert '+dwiInterim_dir+'/dti_tp_unring.mif.gz'
        command = command +' '+dwiInterim_dir+'/dti_tp_unring.nii.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        ##############################
        # Run topup on the data
        print('\n3. Run topup')
        ##############################
        #. Note that this takes hours.

        # Get the total readout time from the json file
        # Find the json files that we still have in the main folder
        dwiInfoJsonFiles = glob.glob(convert_dir+'/*b0_b*.json' , recursive=False)
        topupInfoJsonFiles = glob.glob(convert_dir+'/*_phase_*.json' , recursive=False)
        tmpJson = open(dwiInfoJsonFiles[0])
        JsonData = json.load(tmpJson)
        dwiTotalReadoutTime = JsonData['TotalReadoutTime']

        tmpJson = open(topupInfoJsonFiles[0])
        JsonData = json.load(tmpJson)
        topupTotalReadoutTime = JsonData['TotalReadoutTime']

        # top-up requires an even number of slices: pad if necessary
        # DWI data
        img = nib.load(dwiInterim_dir+'/dti_unring.nii.gz')
        data = img.get_fdata()

        if (data.shape[2]%2==1):
            padded = np.zeros((data.shape[0],data.shape[1],data.shape[2]+1,data.shape[3]))
            padded[0:data.shape[0],0:data.shape[1],0:data.shape[2],0:data.shape[3]] = data
        else:
            padded = data

        img = nib.Nifti1Image(padded, affine=img.affine,header=img.header)
        nib.save(img, dwiInterim_dir+'/dti.nii.gz')

        shutil.copy(dwiInterim_dir+'/dti_unring.bval', dwiInterim_dir+'/dti.bval')
        shutil.copy(dwiInterim_dir+'/dti_unring.bvec', dwiInterim_dir+'/dti.bvec')

        # Topup data
        img = nib.load(dwiInterim_dir+'/dti_tp_unring.nii.gz')
        data = img.get_fdata()

        if (data.shape[2]%2==1):
            padded = np.zeros((data.shape[0],data.shape[1],data.shape[2]+1,data.shape[3]))
            padded[0:data.shape[0],0:data.shape[1],0:data.shape[2],0:data.shape[3]] = data
        else:
            padded = data

        img = nib.Nifti1Image(padded, affine=img.affine, header=img.header)
        nib.save(img, dwiInterim_dir+'/dti_tp.nii.gz')

        # run the fsl topup script
        # 1. (extract total readout time from json files)
        totreadout_main = dwiTotalReadoutTime
        totreadout_topup = topupTotalReadoutTime

        # 2. (merge all B0 from normal and topup acquisitions)
        command = 'dwiextract -bzero'
        command = command +' -fslgrad '+dwiInterim_dir+'/dti.bvec '+dwiInterim_dir+'/dti.bval'
        command = command +' '+dwiInterim_dir+'/dti.nii.gz'
        command = command +' '+dwiInterim_dir+'/b0.nii.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        shutil.copy(dwiInterim_dir+'/dti_tp.nii.gz', dwiInterim_dir+'/b0_tp.nii.gz')

        command = 'mrcat -axis 3 '
        command = command +' '+dwiInterim_dir+'/b0.nii.gz'
        command = command +' '+dwiInterim_dir+'/b0_tp.nii.gz'
        command = command +' '+dwiInterim_dir+'/b0_all.nii.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # 3. (build the corresponding acquisition parameter file)
        acqfile = open(dwiInterim_dir+'/acqparams.txt', 'w')
        b0_img = nib.load(dwiInterim_dir+'/b0.nii.gz')
        b0_tp_img = nib.load(dwiInterim_dir+'/b0_tp.nii.gz')
        for file in range(b0_img.header.get_data_shape()[3]):
            acqfile.write("0 1 0 "+str(totreadout_main)+"\n")
        for file in range(b0_tp_img.header.get_data_shape()[3]):
            # The first B0 tp has the same blib direction as the main DWI file
            if file == 0:
                acqfile.write("0 1 0 "+str(totreadout_main)+"\n")
            else:
                acqfile.write("0 -1 0 "+str(topupTotalReadoutTime)+"\n")
        acqfile.close()

        idxfile = open(dwiInterim_dir+'/index.txt', 'w')
        dti_img = nib.load(dwiInterim_dir+'/dti.nii.gz')
        for file in range(dti_img.header.get_data_shape()[3]):
            idxfile.write("1  ")
        idxfile.close()

        # 4. (run topup)
        # Note that we are using a custom b02b0_hires config file where we 
        # increased the resolution and updated the subsample steps:
        # warpres: 2.5; subsample: 1; smoothing: 0; miter: 25
        command = 'topup --imain='+dwiInterim_dir+'/b0_all.nii.gz'
        command = command +' --datain='+dwiInterim_dir+'/acqparams.txt'
        command = command +' --config='+syntaxDir+'/b02b0_hires.cnf'
        command = command +' --out='+dwiInterim_dir+'/topup'
        command = command +' --fout='+dwiInterim_dir+'/tfield'
        command = command +' --iout='+dwiInterim_dir+'/topuped'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # 5. (average A and P DWIs)
        command = 'fslmaths '+dwiInterim_dir+'/topuped -Tmean '+dwiInterim_dir+'/meanB0_AP'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # 6. (mask with BET)
        command = 'bet '+dwiInterim_dir+'/meanB0_AP '+dwiInterim_dir+'/bet_meanB0_AP -f 0.1 -m -R'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # 7. (eddy)
        #. Note that if you can get eddy_openmp to work on your setup 
        #. you can start to use multiple cores which is highly recommended
        command = 'eddy_openmp --imain='+dwiInterim_dir+'/dti.nii.gz'
        command = command +' --mask='+dwiInterim_dir+'/bet_meanB0_AP_mask.nii.gz'
        command = command +' --acqp='+dwiInterim_dir+'/acqparams.txt'
        command = command +' --index='+dwiInterim_dir+'/index.txt'
        command = command +' --bvecs='+dwiInterim_dir+'/dti.bvec'
        command = command +' --bvals='+dwiInterim_dir+'/dti.bval'
        command = command +' --topup='+dwiInterim_dir+'/topup'
        command = command +' --repol'
        command = command +' --out='+dwiInterim_dir+'/eddy_corr_data'
        command = command +' --nvoxhp=4000 --ol_nstd=3 --slm=linear --mb=1 --fwhm=10,0,0,0,0'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        ##############################
        # N4 bias field correction
        print('\nBias field correction')
        ##############################
        # Combine the eddy corrected data into a mif file
        command = 'mrconvert -fslgrad '+dwiInterim_dir+'/dti.bvec'
        command = command +' '+dwiInterim_dir+'/dti.bval'
        command = command +' '+dwiInterim_dir+'/eddy_corr_data.nii.gz'
        command = command +' '+dwiInterim_dir+'/dwi-preproc.mif.gz'
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Run bias correction
        command = 'dwibiascorrect ants '+dwiInterim_dir+'/dwi-preproc.mif.gz'
        command = command +' '+dwiInterim_dir+'/dwi-preproc_unbiased.mif.gz'
        command = command +' -bias '+dwiInterim_dir+'/bias.mif'
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Create a mask of the bias corrected data
        command = 'dwi2mask '+dwiInterim_dir+'/dwi-preproc_unbiased.mif.gz'
        command = command+' '+dwiInterim_dir+'/mask.mif' 
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Fill the internal holes of the mask
        # First convert it to nifti
        command = 'mrconvert '+dwiInterim_dir+'/mask.mif '+dwiInterim_dir+'/mask.nii.gz' 
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # then fill holes
        command = 'fslmaths '+dwiInterim_dir+'/mask.nii.gz'
        command = command+' -bin -fillh '+dwiInterim_dir+'/mask_filled.nii.gz'
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)
        # Convert back to mif
        command = 'mrconvert '+dwiInterim_dir+'/mask_filled.nii.gz '+dwiInterim_dir+'/mask_filled.mif.gz' 
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)
            
        ##############################
        # Fit the actual tensor model to go from dwi to dti
        print('Fit tensor model')
        ##############################
        # Fit a tensor model to the DWI data where we also try to predict the signal based on the fitted model
        #. This will be used to calculate the raw residual
        command = 'dwi2tensor '+dwiInterim_dir+'/dwi-preproc_unbiased.mif.gz '+dwiInterim_dir+'/dti-tensor.mif.gz -predicted_signal '+dwiInterim_dir+'/dwi_predict.mif'
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # Calculate the residual of the measured signal versus the predicted signal
        command = 'mrcalc '+dwiInterim_dir+'/dwi-preproc_unbiased.mif.gz '+dwiInterim_dir+'/dwi_predict.mif -subtract '+dwiInterim_dir+'/residual.nii.gz'
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        # In the end we want to have the following tensor metrices:
        #. ADC, FA, RD, AD and the main vector maps
        command = 'tensor2metric -fa '+dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-fa.nii.gz -rd '+dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-rd.nii.gz -ad '+dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-ad.nii.gz -adc '+dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-adc.nii.gz -vec '+dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-vec.nii.gz -mask '+dwiInterim_dir+'/mask_filled.nii.gz '+dwiInterim_dir+'/dti-tensor.mif.gz'
        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)
            
        # Clean up
        #. If save intermediate is false we want to remove the entire folder:
        if saveIntermediatedFiles == False:
            shutil.rmtree(dwiInterim_dir, ignore_errors=True)
            
#################################################################################

### Process the ASL scans

In [ ]:
#################################################################################
# Process the ASL data
#################################################################################
# Note that betting these scans is super hard.. So we are ignoring those for now and  
#. are just moving them to the right folder. 
overwrite = True

# Copy the ASL data to the right folder and try to bet it
for i in range(0, len(subjects)):
    print(subjects[i])
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
        
    convert_dir = work_dir+'/dcm2nii'
    scansOfNoInterest_dir = convert_dir+'/scansOfNoInterest'
    anat_dir = work_dir+'/anat'
    perf_dir = work_dir+'/perf'
    
    if ((os.path.exists(perf_dir)) & (overwrite == False)):
        print('Process the ASL data: done (set overwrite to true to recompute)')
    else:
        print('\n Process the ASL data')
        try: 
            shutil.rmtree(perf_dir, ignore_errors=True)
            os.makedirs(perf_dir)
        except:
            os.makedirs(perf_dir)
        # Get the ASL processed data
        #. Note that this is data which is processed manually at the console based
        #. on the raw ASL data. We are not going to process the raw ASL data here
        #. Instead we are just going to copy the processed data and give it the right
        #. BIDS names and folder location:
        processedASLData = glob.glob(scansOfNoInterest_dir+'/*cbf*.*' , recursive=False)
        print(processedASLData)
        for file in processedASLData:
            print(file)
            shutil.copy(file, perf_dir)

        # rename the nifti file:
        old_name = glob.glob(perf_dir+'/*cbf*.nii.gz', recursive=False)[0]  
        new_name =  perf_dir+'/'+subjects[i]+'_'+session+'_asl_desc-cbf.nii.gz'
        os.rename(old_name, new_name)

        # rename the json file:
        old_nameJson = glob.glob(perf_dir+'/*cbf*.json', recursive=False)[0]  
        new_nameJson =  perf_dir+'/'+subjects[i]+'_'+session+'_asl_desc-cbf.json'
        os.rename(old_nameJson, new_nameJson)
         
#################################################################################

In [ ]:
#################################################################################
# Register everything to the MP2RAGE and MNI space
#################################################################################
# The scans that we want to register to the MP2RAGE space are 
#. the QSM image
#. the mean B0 image
#. The scans that we want to register to MNI space are:
#. the mp2rage image
#. the QSM image
#. the mean B0 image
overwrite = True

for i in range(0, len(subjects)):
    print(subjects[i])

    #. BIDS'ish layout: subject, session, modality, data
    #. Where we make a seperate folder for pilot data and final data in the ISLAND folder
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
    
    # Local path settings
    anat_dir = work_dir+'/anat'
    dwi_dir = work_dir+'/dwi'
    perf_dir = work_dir+'/perf'
    mp2rageSpace_dir = work_dir+'/anat_space'
    mniSpace_dir = work_dir+'/mni_space'
    
    if ((os.path.exists(mp2rageSpace_dir)) & (overwrite == False)):
        print('Register to mp2rage and MNI space data: done (set overwrite to true to recompute)')
    else:
        print('\n Register to mp2rage and MNI space')
        try: 
            shutil.rmtree(mp2rageSpace_dir, ignore_errors=True)
            os.makedirs(mp2rageSpace_dir)
            shutil.rmtree(mniSpace_dir, ignore_errors=True)
            os.makedirs(mniSpace_dir)
        except:
            os.makedirs(mp2rageSpace_dir)
            os.makedirs(mniSpace_dir)
        
        # Location of the MNI template (is defined at the top)
        mniTemplate = mniTemplate
        
        # Location of the scans:
        mp2rage = anat_dir+'/'+subjects[i]+'_'+session+'_T1w.nii.gz'
        mp2rageT1map = anat_dir+'/'+subjects[i]+'_'+session+'_T1map.nii.gz'
        mp2rageT1map2MNI = mniSpace_dir+'/'+subjects[i]+'_'+session+'_T1map_2_mni.nii.gz'
        
        # QSM 
        #. Instead of using the calculated QSM image we are going to use the 
        #. magnitude image for the registration as it contains more anatomical info
        #. and then apply the transformation to the actual QSM image. 
        qsm4registration = work_dir+'/bet/'+subjects[i]+'_'+session+'_brain_Chimap.nii.gz'
        qsm = anat_dir+'/'+subjects[i]+'_'+session+'_Chimap.nii.gz'
        qsm2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_Chimap_2_mp2rage.nii.gz'
        qsm2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_Chimap_2_mni.nii.gz'
        
        # DWI.
        #. Similarly for the DTI based metrices we are going to use the mean B0 
        #. image for the registration and then apply the transformation to the 
        #. FA, AD etc
        dwi4registration = dwi_dir+'/intermediateFiles/bet_meanB0_AP.nii.gz'
        dwi2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_meanB0_2_mp2rage.nii.gz'
        dwi2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_meanB0_2_mni.nii.gz'
    
        # The different DTI based metrices and the corresponding output names:
        dtiVec = dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-vec.nii.gz'
        dtiFA = dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-fa.nii.gz'
        dtiRD = dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-rd.nii.gz'
        dtiAD = dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-ad.nii.gz'
        dtiADC = dwi_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-adc.nii.gz'

        dtiVec2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-vec_2_mp2rage.nii.gz'
        dtiFA2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-fa_2_mp2rage.nii.gz'
        dtiRD2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-rd_2_mp2rage.nii.gz'
        dtiAD2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-ad_2_mp2rage.nii.gz'
        dtiADC2mp2rage = mp2rageSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-adc_2_mp2rage.nii.gz'

        dtiVec2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-vec_2_mni.nii.gz'
        dtiFA2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-fa_2_mni.nii.gz'
        dtiRD2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-rd_2_mni.nii.gz'
        dtiAD2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-ad_2_mni.nii.gz'
        dtiADC2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-adc_2_mni.nii.gz'

        # The QSM and DWI inputs have already been betted, we just need to apply the brain mask of the MP2RAGE INV2 to
        #. the T1w image:
        mp2rageMask = work_dir+'/bet/'+subjects[i]+'_'+session+'_brain_T1w-INV2_mask.nii.gz'
        mp2rageBet = anat_dir+'/'+subjects[i]+'_'+session+'_T1w_bet.nii.gz'
        
        # Mask the mp2rage image with the brain mask
        command = 'fslmaths '+mp2rage+' -mas '+mp2rageMask+' '+mp2rageBet

        print(command)
        try:
            subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
            raise subprocess.CalledProcessError(msg)

        
        # 0) Estimate deformation field image (moving) -> MNI (fixed) using ANTs
        # Start registration
        registrationStepsMNI = [1, 2]
        for registrationStep in registrationStepsMNI:
            if registrationStep == 1:
                fixed = ants.image_read(mniTemplate)
                moving = ants.image_read(mp2rageBet)
                #fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)

                mytx0 = ants.registration(fixed = fixed , moving = moving, type_of_transform = 'SyN' , write_composite_transform = True)
                warped_moving = mytx0['warpedmovout']
                #fixed.plot(overlay = warped_moving, title = 'After First Registration', overlay_alpha = 0.75)

            if registrationStep == 2:
                fixed = ants.image_read(mniTemplate)
                moving = warped_moving #ants.image_read(warped_moving)
                #fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)

                mytx1 = ants.registration(fixed = fixed , moving = moving, type_of_transform = 'SyNAggro', write_composite_transform = True)
                warped_moving = mytx1['warpedmovout']
                #fixed.plot(overlay = warped_moving, title = 'After Second Registration Optimization', overlay_alpha = 0.75)

        # Check if the direct registration worked
        fixed = ants.image_read(mniTemplate)
        moving = ants.image_read(mp2rageBet)
        fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)
        warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx0['fwdtransforms'], mytx1['fwdtransforms']])
        fixed.plot(overlay = warpedimage, title = 'After Direct mp2rage 2 mni Registration', overlay_alpha = 0.75)
        
        # And apply to the qMRI T1 map
        fixed = ants.image_read(mniTemplate)
        moving = ants.image_read(mp2rageT1map)
        movingName =  mp2rageT1map2MNI
        warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx0['fwdtransforms'], mytx1['fwdtransforms']])

        fixedNibabel = ants.to_nibabel(fixed)
        warpedimageNibabel = ants.to_nibabel(warpedimage)
        # Where I need to copy the affine matrix of the B0 image because the warpfield does something weird with the matrix
        #. which then interfers with mrview
        image2Nifti = warpedimageNibabel.__class__(warpedimageNibabel.dataobj[:], fixedNibabel.affine, warpedimageNibabel.header)
        image2Nifti.to_filename(movingName)
        
    # 1) Estimate deformation field image (moving) -> MP2RAGE (fixed) using ANTs
        #. After some testing QSM seems to work as long as you stay away from severe 
        #. nonlinear methods (probably would gain improvement by making the mp2rage betting
        #. even better). DWI works well with dense Rigid and then followed by a single nonlinear step
        modalities2mp2rage = ['qsm', 'dwi']
        for modality in modalities2mp2rage:
            print(modality)
            if modality == 'qsm':
                movingImageReg = qsm4registration
                movingImage= qsm
                outputImageMP2RAGE = qsm2mp2rage
                outputImageMNI = qsm2mni
                additionalMovingImages = []
                typeOfTransform1 = 'Affine'
                metric = 'GC'
            elif modality == 'dwi':
                movingImageReg = dwi4registration
                movingImage = dwi4registration
                outputImageMP2RAGE = dwi2mp2rage
                outputImageMNI = dwi2mni
                additionalMovingImages = [dtiVec, dtiFA, dtiRD, dtiAD, dtiADC]
                additionalMovingImagesNamesMP2RAGE = [dtiVec2mp2rage, dtiFA2mp2rage, dtiRD2mp2rage, dtiAD2mp2rage, dtiADC2mp2rage]
                additionalMovingImagesNamesMNI = [dtiVec2mni, dtiFA2mni, dtiRD2mni, dtiAD2mni, dtiADC2mni]
                typeOfTransform1 = 'Affine'
                metric = 'mattes'

            # Start registration
            #. within subject, single step registration seemed to work better. 
            registrationStepsMP2RAGE = [1] 
            for registrationStep in registrationStepsMP2RAGE:
                if registrationStep == 1:
                    fixed = ants.image_read(mp2rageBet)
                    moving = ants.image_read(movingImageReg)
                    #fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)

                    mytx2 = ants.registration(fixed = fixed , moving = moving, type_of_transform = typeOfTransform1, aff_metric = metric, write_composite_transform = True)
                    warped_moving = mytx1['warpedmovout']
                    #fixed.plot(overlay = warped_moving, title = 'After First Registration', overlay_alpha = 0.75)

            # Check if the direct registration 2 mp2rage worked
            fixed = ants.image_read(mp2rageBet)
            moving = ants.image_read(movingImageReg)
            fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)
            warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx2['fwdtransforms']])
            fixed.plot(overlay = warpedimage, title = 'After Direct Registration MP2RAGE', overlay_alpha = 0.75)
            
            # 3) Save the ants image to nifti
            # Export the ANTSimage to a nifti via nibabel
            fixedNibabel = ants.to_nibabel(fixed)
            warpedimageNibabel = ants.to_nibabel(warpedimage)
            # Where I need to copy the affine matrix of the B0 image because the warpfield does something weird with the matrix
            #. which then interfers with mrview
            movingimage2Nifti = warpedimageNibabel.__class__(warpedimageNibabel.dataobj[:], fixedNibabel.affine, warpedimageNibabel.header)
            movingimage2Nifti.to_filename(outputImageMP2RAGE)
            
            # Check if the direct registration 2 mni worked
            fixed = ants.image_read(mniTemplate)
            moving = ants.image_read(movingImage)
            fixed.plot(overlay = moving, title = 'Before Registration', overlay_alpha = 0.75)
            warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx2['fwdtransforms'], mytx0['fwdtransforms'], mytx1['fwdtransforms'] ]) 
            fixed.plot(overlay = warpedimage, title = 'After Direct Registration MNI', overlay_alpha = 0.75)
            
            # 3) Save the ants image to nifti
            # Export the ANTSimage to a nifti via nibabel
            fixedNibabel = ants.to_nibabel(fixed)
            warpedimageNibabel = ants.to_nibabel(warpedimage)
            # Where I need to copy the affine matrix of the fixed image to the moving image because the warpfield does something weird with the matrix
            #. which then interfers with mrview
            movingimage2Nifti = warpedimageNibabel.__class__(warpedimageNibabel.dataobj[:], fixedNibabel.affine, warpedimageNibabel.header)
            movingimage2Nifti.to_filename(outputImageMNI)
          
            # Apply the transformation matrix to the different images that still need to be 
            #. transformed to mp2rage Space
            for ii in range(0, len(additionalMovingImages)):
                fixed = ants.image_read(mp2rageBet)
                moving = ants.image_read(additionalMovingImages[ii])
                movingName = additionalMovingImagesNamesMP2RAGE[ii]
                try:
                    warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx2['fwdtransforms']])
                except:
                    # Some DTI images are 4D and thus you need to change the imagetype
                    warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx2['fwdtransforms']], imagetype = 3)
                fixedNibabel = ants.to_nibabel(fixed)
                warpedimageNibabel = ants.to_nibabel(warpedimage)
                # Where I need to copy the affine matrix of the fixed image to the moving image because the warpfield does something weird with the matrix
                #. which then interfers with mrview
                image2Nifti = warpedimageNibabel.__class__(warpedimageNibabel.dataobj[:], fixedNibabel.affine, warpedimageNibabel.header)
                image2Nifti.to_filename(movingName)
                
            #. Any image that need to be transformed to MP2RAGE also needs to go to MNI Space
            for ii in range(0, len(additionalMovingImages)):
                fixed = ants.image_read(mniTemplate)
                moving = ants.image_read(additionalMovingImages[ii])
                movingName = additionalMovingImagesNamesMNI[ii]
                try:
                    warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx0['fwdtransforms'], mytx1['fwdtransforms'], mytx2['fwdtransforms']])
                except:
                    # Some DTI images are 4D and thus you need to change the imagetype
                    warpedimage = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=[mytx0['fwdtransforms'], mytx1['fwdtransforms'], mytx2['fwdtransforms']], imagetype = 3)
                fixedNibabel = ants.to_nibabel(fixed)
                warpedimageNibabel = ants.to_nibabel(warpedimage)
                # Where I need to copy the affine matrix of the B0 image because the warpfield does something weird with the matrix
                #. which then interfers with mrview
                image2Nifti = warpedimageNibabel.__class__(warpedimageNibabel.dataobj[:], fixedNibabel.affine, warpedimageNibabel.header)
                image2Nifti.to_filename(movingName)
#################################################################################

In [ ]:
#################################################################################
# Extract qMRI metrics from the different scans in MNI space
#################################################################################
# We have a number of qMRI metrices in MNI space and we now want to know what the 
#. mean and std values are of a number of regions. The regions are based on an 
#. MNI atlas: http://www.brainnetome.org/
#. Note that the MNI template used for this atlas is not identical in dimensions compared
#. to the MNI template used in the pipeline. So we have to copy the dimensions to
#. the atlas file. 
#. This was done with the flirt:
#.   flirt -in BN_Atlas_246_1mm.nii -ref avg152T1_brain.nii.gz -out BN_Atlas_246_1mm_fsl.nii.gz 
#.      -bins 256 -cost corratio -searchrx 0 0 -searchry 0 0 -searchrz 0 0 -dof 6 
#.      -schedule /usr/local/fsl/etc/flirtsch/sch3Dtrans_3dof  -interp nearestneighbour


# Finding the atlases, templates etc
fslImageStats = fsl.ImageStats() 
mniAtlasLabels = mniAtlasLabels # defined at the beginning
mniAtlas = mniAtlas # defined at the beginning
mniAtlasHeader = syntaxDir+'/MNIAtlas/BN_Atlas_246_1mm_fsl.nii'
dfmniAtlasLabels = pd.read_csv(mniAtlasLabels, header = None, sep = ' ') # Read in the labels 
dfmniAtlasLabels.columns = ['Index', 'Structure', 'r', 'g', 'b', 'notUsed'] # and make the labels nice

# Create seperate masks per structure based on the MNI atlas
print('Splitting masks')
for i in range(0, 247):
    structure = dfmniAtlasLabels['Structure'].iloc[i]
    structure = structure.replace(r'/', '-')
    masksLabel = syntaxDir+'/MNIAtlas/'+str(structure)+'.nii.gz'
    command = 'fslmaths '+mniAtlasHeader+' -thr '+str(i)+' -uthr '+str(i)+' '+masksLabel
    try:
        subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        msg = 'execution failed (error code '+str(e.returncode)+')\n Output: '+str(e.output)
        raise subprocess.CalledProcessError(msg)
        
# Get all the seperate masks as a list
MNIMasks = syntaxDir+'/MNIAtlas'
listOfStructures = glob.glob(MNIMasks+'/*.nii.gz')

# For all subjects, contrasts and structures we want to have the mean and std value in an excel file    
print('Extracting qMRI values')
qMRIValueExtraction = []
for i in range(0, len(subjects)):
    print(subjects[i])
    
    # Local path settings
    if session == 'pilot':
        work_dir = processedDataDir+'/pilot/'+subjects[i]+'/'+session
    else:
        work_dir = processedDataDir+'/ISLAND/'+subjects[i]+'/'+session
    anat_dir = work_dir+'/anat'
    dwi_dir = work_dir+'/dwi'
    perf_dir = work_dir+'/perf'
    mp2rageSpace_dir = work_dir+'/anat_space'
    mniSpace_dir = work_dir+'/mni_space'

    # Which files in MNI space do we actually want to extract the values from?
    mp2rageT1map2MNI = mniSpace_dir+'/'+subjects[i]+'_'+session+'_T1map_2_mni.nii.gz'
    qsm2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_Chimap_2_mni.nii.gz'    
    dtiVec2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-vec_2_mni.nii.gz'
    dtiFA2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-fa_2_mni.nii.gz'
    dtiRD2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-rd_2_mni.nii.gz'
    dtiAD2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-ad_2_mni.nii.gz'
    dtiADC2mni = mniSpace_dir+'/'+subjects[i]+'_'+session+'_dwi_desc-adc_2_mni.nii.gz'
    # Lets make a list of those files and provide names that we are going to use in the excel sheet
    contrastsImage = [mp2rageT1map2MNI, qsm2mni, dtiVec2mni, dtiFA2mni, dtiRD2mni, dtiAD2mni, dtiADC2mni]
    contrastNames = ['T1', 'QSM', 'DTI-Vec', 'DTI-FA', 'DTI-RD', 'DTI-AD', 'DTI-ADC']
    # For all the contrast images and all anatomical structures included in the atlas extract
    #. the mean and std values and put them in a list:
    for j in range(0, len(contrastsImage)):
        print(contrastsImage[j])
        for structure in listOfStructures:
            structureName = structure.split('/')[-1].split('.')[0]
            valueExtract = fslImageStats.run(in_file = contrastsImage[j],  mask_file = structure, op_string = '-k %s -m -s').outputs.out_stat
            qMRIValueExtraction.append({'Subject': subjects[i], 'Contrast':contrastNames[j], 'Structure': structureName, 'Mean': valueExtract[0], 'Std':valueExtract[1]})

# Export the qMRI values to a dataframe and then to an excel file:
dfContrastValues = pd.DataFrame(qMRIValueExtraction)
dfContrastValues.to_excel(processedDataDir+'/ISLAND_BNAtlas_qMRI-values.xlsx')
#################################################################################